In [ ]:
import pandas as pd
import ipcalc
import parse_functions as pf
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
import user_agents as ua
from helpers import filter_other, autopct_format
import calendar

# Vorbereitung: Einlesen der Daten

_Vorstellung: Flo_

- Für die Analsyse müssen die Daten der `acces.log`-Datei eingelesen werden!

## Schwierigkeit

- Größe des Datensatzes: 10.365.152 Log Einträge (über 10 Milionen)
- keine eindeutiges Trennzeichen in den Daten vorhanden
- kein von Pandas vorgefertigter Import für Log Dateien

**So sehen die Anfragen aus:**
```
31.56.96.51 - - [22/Jan/2019:03:56:16 +0330] "GET /image/60844/productModel/200x200 HTTP/1.1" 200 5667 "https://www.zanbil.ir/m/filter/b113" "Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build/HuaweiALE-L21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.158 Mobile Safari/537.36" "-"

31.56.96.51 - - [22/Jan/2019:03:56:16 +0330] "GET /image/61474/productModel/200x200 HTTP/1.1" 200 5379 "https://www.zanbil.ir/m/filter/b113" "Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build/HuaweiALE-L21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.158 Mobile Safari/537.36" "-"
```

## Lösung

- Nutzung des Read_CSV Befehls angewandt auf die `acces.log`-Datei
- Anwendung möglich durch die Angabe des Seperators mitthilfe einer Regular Expression ```sep=r'\s(?=(?:[^"]*"[^"]*")*[^"]*$)(?![^\[]*\]\s)'```
- Nach eindeutiger zuordnung des Trennzeichens können Daten eingelesen werden!

**Nachteil:** Die Ladezeit ist aufgrund der Größe des Datensatzes relativ lang (ca. 10-15 Minuten)

In [ ]:
# Source: https://mmas.github.io/read-apache-access-log-pandas
access_log = pd.read_csv(
    'data/access.log',
    sep=r'\s(?=(?:[^"]*"[^"]*")*[^"]*$)(?![^\[]*\]\s)',
    engine='python',
    na_values='-',
    header=None,
    usecols=[0, 3, 4, 5, 6, 7, 8],
    names=['ip', 'time', 'request', 'status', 'size', 'referer', 'user_agent'],
    converters={'time': pf.parse_datetime,
                'request': pf.parse_str,
                'status': int,
                'size': int,
                'referer': pf.parse_str,
                'user_agent': pf.parse_str},
    on_bad_lines='warn')

access_log.head()


## Umwandlung in CSV Datei

Nachdem Daten eingelesen wurden diese direkt als CSV-Datei "zwischenspeichern"

**Vorteil:** Verkürzt die Ladezeit auf ca. 3-4 Minuten

- CSV Datei kann genutzt werden um die Daten erneut einzulesen
- Das Einladen über die strukturierte CSV Datei geht deutlich schneller als laden der Daten aus der `acces.log`-Datei

In [ ]:
# Daten in einer CSV Datei speichern
filename = "data/acces_log.csv"
access_log.to_csv(filename, index=False)

## Daten aus CSV laden
- Die Daten aus der zuvor genrierten CSV Datei laden. 

**Wichtig:** Es muss zuvor die CSV Datei generiert werden, indem mindestens einmal die Daten aus der `acces.log`-Datei eingelesen wurden!
- dieser Schritt wird also erst bei der mehrfachen Ausführung des Notebooks sinnvoll! 

In [ ]:
# Daten aus CSV laden
filename = "data/acces_log.csv"
access_log = pd.read_csv(filename)

access_log.head()

In [ ]:
# Erzeuge eine Kopie der Daten als "Backup" 
access_log_backup = access_log.copy()
len(access_log_backup)

## Daten konvertieren & normalisieren
Die in der CSV hinterlegten Daten in die benötigten Datentypen überführen

- time: soll als Datum im Pandas Dataframe hinterlegt und entsprechend Normalisiert werden!

In [ ]:
# Typen konvertieren
access_log['time'] = pd.to_datetime(access_log['time'])

# Daten normalisieren
access_log['time'] = access_log['time'].dt.tz_convert('UTC')

# Aufgabe 1: Beliebtestes Produkt

_Vorstellung: Tom_

> Analysieren Sie welche Produkte beliebt sind. Entwickeln Sie dazu eine Definition eines beliebten Produktes. Stellen Sie die Ergebnisse anschaulich da.

## Definition

> Das Produkt mit dein meisten Aufrufen auf dem Webserver


## Lösungsweg

In [ ]:
access_log.loc[access_log['request'].str.contains('/product/') == True].value_counts(access_log['request']).head()

### Ergebnis

> Das Produkt _55XTU625-Ultra-HD_ mit der ID _33309_ wurde am häufigsten aufgerufen.

**Oder?**

## Lösungsweg II

Es gibt mehrere URLs pro Produkt

In [ ]:
def extract_product_id(request):
  m = re.search(r"\/product\/(\d+)\/", str(request))
  if m:
    return m.group(1)
  return None

access_log['product_id'] = access_log['request'].map(extract_product_id)
access_log.head()

In [ ]:
most_viewed_products = access_log.loc[access_log['product_id'] != None].value_counts(access_log['product_id'])
most_viewed_products = most_viewed_products[0:10]
most_viewed_products.plot(kind='barh')
most_viewed_product_id = most_viewed_products.keys()[0]
print(most_viewed_products[0], 'x', 'ID:', most_viewed_product_id)

access_log.loc[access_log['product_id'] == most_viewed_product_id].value_counts(access_log['request'])

### Ergebnis II

> Das beliebteste Produkt ist #33953, das _Galaxy A7 Dual 128GB_

# Aufgabe 2

> Untersuchen Sie den Datensatz auf weitere Auffälligkeiten.


## Basisinformationen durch Pandas-Befehle
- Pandas bietet bereits vordefinierte Befehle um einfache Informationen über die Daten zu ermitteln
- Mit den Befehlen zu beginnen, hilft dabei die Daten ein erstes mal zu erkunden!
- Speziell bei numerrischen Daten erhält man bereits eine Reihe an spannenden statistischen Informationen
- Auch für Daten wie die des Logs, lohnt es sich kurz beide Befehle anzugucken!

### Befehl `.info()`
- Übersicht über die Spalten
- Angabe zu den Datentypen der Spalten

In [ ]:
access_log.info()

### Befehl `.describe()`
- Anzahl der Einträge
- wie viele einzigartige Einträge
- höchste Zahl
- Durchschintt
- weitere statistische Angaben!

--> Statistische Basisinformationen <br>
--> Parameter `include = 'all'` notwendig, dass auch nicht numerische Daten in die Übersicht mit übernommen werden



In [ ]:
access_log.describe(include = 'all')

## Daten anzeigen

- Zum Anzeigen der Daten können unterschiedliche Methoden genutzt werden
- Anzeigen durch Methoden: ` .head(x) | .tail(x) ` gibt die ersten / letzten x Zeilen aus
- indizierter Zugriff wie aus Python bekannt `acces_log[10000:10200]`

In [ ]:
access_log.head(5)

In [ ]:
access_log[10000:10200]

## Einfache Analyse der Anzahl einzelner Features

- Zählen der Features durch `.value_counts("feature")`
  - ermitteln welche IP Adresse die meisten Anfragen stellt
  - Verwendung auch in Aufgabe 1 (Welches Produkt wurde am meisten angefragt)
  - Von welchen Seiten kamen die Leute zuvor hauptsächlich?

In [ ]:
# IP-Adressen mit den meisten Aufrufen 
access_log.value_counts("ip").head()

### Referer
- Analyse von welchen Seiten die Nutzer gekommen sind (Wo war der Nutzer zuvor?)
- Einfache Anzahlen können direkt visualisiert und als Plot dargestellt werden (Unterschiedliche Darstellungsmöglichkeiten (Bar, Pie...))


In [ ]:
referer_counts = access_log.loc[access_log['referer'] != None]['referer'].value_counts()
referer_counts.head(10).plot(kind="barh")

In [ ]:
# Pie Plot mit allen weiteren Links
referer_counts = access_log.loc[access_log['referer'] != None]['referer'].value_counts()
referer_counts = filter_other(referer_counts, 0.01)
referer_counts.plot(kind='pie', title='Request Methods', ylabel='Method', autopct=autopct_format(referer_counts))

# Auswertung Referer

1. Sehr viele der Referer Eintäge haben was mit `Zanbil` zu tun (Platz 1-4 % 6-10)
   - **Unsere Schlussfolgerung:** Der Webserver-Log wird von der Seite Zanbil.ir stammen, da sehre viele der Aufrufe von der gleichen Website stammen  
   - Dies passiert wenn die Nutzer zwischen der Seite hin und her springen
2. Nur `Google.com` hat es noch in die Top 10 der Referer einträge geschafft! Vermutlich, da es die Meist genutzte Suchmaschine ist, über die die Website entsprechend gefunden werden konnte

## Datenergänzung / Datenaufbereitung zur Analyse

### Timestamps

- Zur Analyse des Timestamps werden einzelne Spalten für Wochentag (numerisch), Stunde und Datum erstellt
- Die neu erstellten Features können für Untersuchungen zu den Aufrufe nach Tageszeit / Tagen verwendet werden

In [ ]:
access_log['date'] = access_log['time'].dt.date
access_log['weekday_n'] = access_log['time'].dt.weekday
access_log['weekday'] = access_log['weekday_n'].map(lambda wd: calendar.day_name[wd])
access_log['hour'] = access_log['time'].dt.hour

access_log.head()

## Analyse der herausgearbeiteten Timestamp Eigenschaften 

- Analyse der Aufrufe je Tag
- Analyse der Aufrufe je Stunden

Darstellung beider Daten als Balkendiagramm

In [ ]:
# Analyse der Tage (Zugriffstage)
days_count = access_log.value_counts('date')
print(days_count)
days_count.sort_index().plot(kind='bar')

## Auswertung Tagesanzahl

- Die Log-Datei enthält Informationen zu 5 Tagen
- Die ersten beiden Tage haben eine leicht höhere Anzahl an Anfragen, ansonsten aber eine relativ gleiche Verteilung der Last über alle Tage

In [ ]:
# Analyse der Stunden (Zugriffszeiten)
hours_count = access_log.value_counts('hour')
print(hours_count.head())
hours_count.sort_index().plot(kind='bar')

## Auswertung Stundenanalyse (Zugriffszeiten)

- Klares Tief in der Nacht zu erkennen
- Morgens ab 5 Uhr steigt die Anzahl an Aufrufen rapide
- Hochpunkt ist zwischen 8:00 und 9;00 Uhr morgens
- Über den Nachmittag zum Abend hin sinken die Aufrufe wieder deutlich

### Heatmap Tageszeit

_Vorstellung: Flo_

- Alternative visualisierung der Tageszeiten

In [ ]:
daytime_access = access_log.groupby(['hour']).size().to_frame(name = 'count').reset_index()

fig, ax = plt.subplots(1, 1, figsize = (5, 3))
df_wide = daytime_access.pivot_table(columns='hour', values='count', aggfunc=lambda x:x)
heatmap = sns.heatmap(df_wide, linewidths=1.0,ax=ax)

ax.set_xlim(0, 23)
ax.set_ylim(0, 1)

heatmap.set_title('Aufrufe nach Tageszeit')
heatmap.set_xlabel('Uhrzeit')


Die Wochentage sind folgendermaßen ablesbar:

| Nr. | Wochentag |
| --- | --- |
| 0 | Mo |
| 1 | Di |
| 2 | Mi |
| 3 | Do |
| 4 | Fr |
| 5 | Sa |
| 6 | So |

In [ ]:
daytime_access = access_log.groupby(['weekday_n', 'hour']).size().to_frame(name = 'count').reset_index()

fig, ax = plt.subplots(1, 1, figsize = (5, 3))
df_wide = daytime_access.pivot_table(index='weekday_n',columns='hour',values='count', aggfunc=lambda x:x)
heatmap = sns.heatmap(df_wide, linewidths=1.0,ax=ax)

ax.set_xlim(0, 23)
ax.set_ylim(0, 6)
# ax.set_yticks(range(0, 7), ['Mo', 'Di', 'Mi', 'Do', 'Fr', 'Sa', 'So'])

heatmap.set_title('Aufrufe nach Tag/Uhrzeit')
heatmap.set_xlabel('Uhrzeit')
heatmap.set_ylabel('Wochentag')

## Request Methods

Herausarbeiten welche unterschiedlichen Requests an den Server gestellt wurden

Die folgenden Request Methoden gibt es:  
`GET`, `HEAD`, `POST`, `PUT`, `DELETE`, `CONNECT`, `OPTIONS`, `TRACE` und `PATCH`

Hierzu wird auf die Spalte `request` die Methode mit der Funktion `extract_method(str)` extrahiert  
Die Extraktion erfolgt mittels der Regular Expression `^[A-Z]+`, die alle Großbuchstabenn vom Beginn des Request ausliest.

In [ ]:
def extract_method(request):
  "Method to extract the HTTP-Method for a Request"
  m = re.match(r'^[A-Z]+', str(request))
  if m:
    return m.group()
  return None

access_log['method'] = access_log['request'].map(extract_method)
access_log.head()

In [ ]:
method_counts = access_log.loc[access_log['method'] != None]['method'].value_counts()

# Ausgabe der Methods
print(method_counts)

# Plot
method_counts_other = filter_other(method_counts, 0.005)
method_counts_other.plot(kind='pie', title='Request Methods', ylabel='Method', autopct=autopct_format(method_counts_other))

In [ ]:
method_counts_without_get = method_counts.drop(labels = 'GET')
method_counts_without_get.plot(kind='bar', title='Request Methods (excl. GET)', ylabel='Requests')

## Auswertung RequestMethoden

- Sehr hohe Anzahl von Get Requests 98.3%
- Es existieren aber auch andere Reqeust
- 1.3% Post Requests
- Weitere Requests: HEAD; OPTIONS; CONNECT; G; E; PROPFIND

In [ ]:
post_requests = access_log.loc[access_log['method'] == 'POST']
post_requests.head()

## User Agents, Browser & OS

- Browser senden `User-Agent` mit.
- Beispiel:
  - `User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36`
  - Chrome `113.0.0.0` mit Windows 10, 64-Bit
- Diese Daten können wir analysieren!

In [ ]:
# Alternativ: CSV (unten)
agents = access_log['user_agent'].map(lambda agent: ua.parse(str(agent)))
access_log['browser_family'] = agents.map(lambda agent: agent.browser.family)
access_log['os_family'] = agents.map(lambda agent: agent.os.family)
access_log['device_family'] = agents.map(lambda agent: agent.device.family)
access_log['device_brand'] = agents.map(lambda agent: agent.device.brand)
access_log['device_model'] = agents.map(lambda agent: agent.device.model)
access_log['is_mobile'] = agents.map(lambda agent: agent.is_mobile)
access_log['is_pc'] = agents.map(lambda agent: agent.is_pc)
access_log['is_bot'] = agents.map(lambda agent: agent.is_bot)

access_log.head()

In [ ]:
# Zwischenstand Als CSV speichern
filename = "data/acces_log_user_agent.csv"
access_log.to_csv(filename, index=False)

In [ ]:
# Laden aus CSV (dauert keine 6 Minuten)
filename = "data/acces_log_user_agent.csv"
access_log = pd.read_csv(filename)
# access_log = access_log.drop(["Unnamed: 0"], axis=1)
access_log.head()

### Browser-Family

Der verwendete Browser wie Chrome, Firefox, ...

> Viele Nutzer sind mit Chrome (oder Chrome Mobile) unterwegs (ca. 45%!)

In [ ]:
browser_family_counts = access_log['browser_family'].value_counts()

browser_family_counts = filter_other(browser_family_counts)

browser_family_counts.plot(kind='pie', title='Browsers', ylabel='Browser', autopct=autopct_format(browser_family_counts))

### Betriebssysteme

> Hauptsächlich Windows- und Android-Nutzer.

In [ ]:
os_family_counts = access_log['os_family'].value_counts()

os_family_counts = filter_other(os_family_counts)

os_family_counts.plot(kind='pie', title='OS', ylabel='OS', autopct=autopct_format(os_family_counts))

### Bots

> 10,7% der Besucher sind Bots

Bots könnten aber auch "normale" User-Agents nutzen.

In [ ]:
bot_counts = access_log['is_bot'].value_counts()

bot_counts.plot(kind='pie', title='Bots', ylabel='Bots', autopct=autopct_format(bot_counts))

### Bots und IP-Adressen

Wir können einzelne IP-Adressen nach Bots gruppieren

In [ ]:
bot_access_log = access_log.loc[access_log['is_bot']]

bot_access_log.value_counts(['ip', 'browser_family'])

### Reevaluation von Aufgabe 1

Ändert sich das Ergebnis, wenn wir Bots ausschließen?

In [ ]:
# Ergebnis von vorher
print('Ergebnis von vorher')
print(most_viewed_products[0], most_viewed_products.keys()[0])

print()

# Neues Ergebnis (ohne Bots)
print('Neues Ergebnis')
most_viewed_products2 = access_log.loc[access_log['is_bot'] != True].loc[access_log['product_id'] != None].value_counts(access_log['product_id'])
most_viewed_product2_id = int(most_viewed_products2.keys()[0])
most_viewed_product2_count = most_viewed_products2[most_viewed_product2_id]
print(most_viewed_product2_count, most_viewed_product2_id)

print()

# Evaluate
if int(most_viewed_product_id) == most_viewed_product2_id:
    print('Gleiches Ergebnis')
else:
    print('Ergebnis verändert')
print('Differenz:', most_viewed_products[0] - most_viewed_product2_count, 'Bot-Aufrufe')

## Statusmeldungen / Fehler im Detail
_Vorstellung: Tom_  

- Untersuchung welcher Status im Reqeust mitgesendet wurde
- Analyse der Erfolgsrate der Requests
- Detailliertere Betrachtung der Fehlgeschlagenen Requests (Requests die Status >= 400 haben)

In [ ]:
status_counts = access_log['status'].value_counts()

# Other Zusammenfassen!
status_counts = filter_other(status_counts, 0.01)

status_counts.plot(kind='pie', title='Status', ylabel='Status Code', autopct=autopct_format(status_counts))

In [ ]:
error_counts = access_log['status'].map(lambda status: status >= 400).value_counts()

error_counts.plot(kind='pie', title='Failed requests', ylabel='Error?', autopct=autopct_format(error_counts))

## Auswertung der Statuscode und Reqeust Analyse

- Sehr viele Codes sind `200 OK` (92.5%)
- Gringe Quote an fehlerhaften Requests (1.7%)

## Detailliertere Betrachtung der Errors:
- Analyse welche Fehler naufgetreten sind

In [ ]:

error_logs = access_log.loc[access_log['status'] >= 400]

error_code_counts = error_logs['status'].value_counts()

error_code_counts = filter_other(error_code_counts, 0.01)

error_code_counts.plot(kind='pie', title='Error', ylabel='Error Code', autopct=autopct_format(error_code_counts))

### Auswertung Fehlercodes
- 404 (59.1%)   | Not Found
- 499 (28.6%)   | Client Closed Request  
  - nginx, wenn Client Verbindungschließt, wärhrend Anforderungen verarbeitet werden  
  - Seite anscheintent zumindest Teilweise duch nginx umgesetzt.
- 500 (8%)      | Internal Server Error
- 403 (3.2% )   | Forbidden

## Korrelationsanalyse der Daten

Im folgenden wird eine Korrelationsanalyse auf den Daten ausgeführt um zu prüfen, ob einzelne Features miteinander korrelieren.

**Definition Korrelation:** Eine Korrelation beschreibt eine Beziehung zwischen zwei oder mehreren Merkmalen (Zusammenhang zweier Variablen)

- **Anmerkung:** Aktuell liegen wenig Numerische Daten vor! -> Korrelation werden jedoch für numerische Daten zu berechnen
- **Vorgehen:** Die nicht numerischen Daten die mit untersucht werden sollen in numerische Daten umwandeln.

### Begründung Korrelationsanalyse

- Untersuchung auf weitere Datenzusammenhänge die vorliegen könnten, aber noch nicht bekannt sind.
- **Beispiel:** Gibt es einen Zusammenhang zwischen der Tageszeit und Post Reqeusts?

In [ ]:
# Korrelationen auf den Daten ohne Ergänzung
fig, ax = plt.subplots(figsize=(5,4))
sns.heatmap(access_log.corr(numeric_only=True), annot=True, cmap="PuOr", fmt=".1f", vmin=-1, vmax=1, linewidths=0.5)

## Ergänzung um weitere Features (Numerische Umwandlung)
- Damit andere Features integriert werden können müssen sie entsprechend in Numerische Features umgewandelt werden können

### Vorgehen

- **Umwandlung des Method-Feature**
  - ```pd.get_dummies()``` Methode um aus einem Feature entsprechend mehrere Numerische Spalten zu erzeugen
  - Jeder Eintrag des Features erhällt eine eigene Spalte, in der das entsprechende Feature mit ```true``` und ```false``` gekennzeichnet ist.  
<br>
- **Umwandlung des IP Feature**
  - IP String abbilden auf eine Numrerischen Wert durch `ip_to_int` Methode

In [ ]:
# Ergänzen um die Dummies Einträge für Method
methods_numeric = pd.get_dummies(access_log['method'])
methods_numeric

In [ ]:
# Anwendung der get_dummies() Methode auf unseren Datensatz für das Method Festrue | Kann nur einmal ausgeführt werden!
methods_numeric = methods_numeric.drop(['CONNECT','E','G','OPTIONS','PROPFIND'], axis=1)

access_log = access_log.join(methods_numeric)
access_log.head()

In [ ]:
# Umwandeln der IP Adresse in ein Numerischen Feature
access_log['ip_n'] = pf.ip_to_int(access_log['ip'])

In [ ]:
# Erneute Korrelationsanalyse
sns.heatmap(access_log.corr(numeric_only=True), annot=True, cmap="PuOr", fmt=".1f", vmin=-1, vmax=1, linewidths=0.5)

## Korrelationanalyse | Berücksichtigen der ID:
- Da die ID nicht bei allen Requests vorliegt, kann diese Analyse nur auf einen geringeren Teil der Daten angewandt werden
- Es werden nur die Reqeusts genommen, bei denen eine ID vorliegt
- das ID Attribute muss hier zusätzlich noch zum Int gewandelt werden!

Anschließend kann eine erneute Korrelationsanalyse durchgeführt werden, in der die Product ID berücktischtigt wird!

In [ ]:
# Analyse der Korrelationen mit Product ID
access_log_copy = access_log.copy()

mask = access_log_copy["product_id"].notnull()
mask.value_counts()
access_log_copy = access_log_copy.loc[mask]
access_log_copy["product_id"] = access_log_copy["product_id"].astype(int)
access_log_copy

In [ ]:
sns.heatmap(access_log_copy.corr(numeric_only=True), annot=True, cmap="PuOr", fmt=".1f", vmin=-1, vmax=1, linewidths=0.5)

In [ ]:
sns.heatmap(access_log.corr(numeric_only=True), annot=True, cmap="PuOr", fmt=".1f", vmin=-1, vmax=1, linewidths=0.5)

## Auswertung Korrelationsanalyse:

**Korrelationmatrizen im Vergleich:**
- Es gibt unterschiede zwischen den Korrelationsanalysen!  
--> Dies lässt sich darauf zurückführen, dass bei der Analyse, die die Product ID beinhaltet eine deutlich geringere Datenmenge vorliegt!


**Korrelationen ohne Product ID:**
- Es konnten keine unerwarteten Korrelationen
- Die Höchstwerte liegen vor zwischen `GET` und `POST` als auch `is_pc` und `is_mobile`  
--> dies ist darauf zurückzuführen, da dies "beinar" binäre Features sind, bei denen nur sehr wenig andere Werte vorkommen.  
--> Somit ist es häufig der der jeweils andere Wert!


**Korrelationen mit Product ID:**
- Auffällige Änderungen: Feature `Status` (es scheint leichte Korrelation zwischen Status und is_mobile zu geben)



## Fazit:
- Erster Grundüberblick über die Korrelationen gesammelt
- Detailliertere Ausarbeitungen könnten folgen